In [1]:
import pprint

""" Import the GreenTranslator API wrapper. """
from greentranslator.api import GreenTranslator

""" Create an instance. """
translator = GreenTranslator ()

In [6]:
"""
> Environmental-Exposures-KS:
  This endpoint enumerates available coordinates.
  It's for prototyping purposes since we have a limited data set. """
exposure = translator.exposures. \
    get_by_coordinates (exposure_type = 'pm25',
                        latitude      = '',
                        longitude     = '',
                        radius        = '0')
pprint.pprint (exposure, indent=2)
    

None


In [7]:
"""
> Environmental-Exposures-KS (Scores):
  Given an exposure type (pm25/o3), a date range, and a location, 
  return timeframes with corresponding exposure scores.
"""
scores = translator.exposures. \
    get_scores (exposure_type = 'pm25',
                start_date = '2010-01-07',
                end_date = '2010-01-31',
                exposure_point = '35.9131996,-79.0558445')
pprint.pprint (scores[:3], indent=2)

TypeError: 'NoneType' object is not subscriptable

In [12]:
"""
> Environmental-Exposures-KS (values):
  Given an exposure type, a date range, and a location, return time bounded exposure values.
"""
values = translator. \
    get_exposures(). \
    get_exposure_values (exposure_type = 'pm25',
                        start_date = '2010-01-07',
                        end_date = '2010-01-31',
                        exposure_point = '35.9131996,-79.0558445')
pprint.pprint (values[:3], indent=2)

[ {'end_time': datetime.datetime(2010, 1, 7, 23, 0, tzinfo=tzutc()),
 'exposure_type': 'pm25',
 'latitude': '35.9131996',
 'longitude': '-79.0558445',
 'start_time': datetime.datetime(2010, 1, 7, 0, 0, tzinfo=tzutc()),
 'units': 'ugm3',
 'value': '17.7199974060059'},
  {'end_time': datetime.datetime(2010, 1, 8, 23, 0, tzinfo=tzutc()),
 'exposure_type': 'pm25',
 'latitude': '35.9131996',
 'longitude': '-79.0558445',
 'start_time': datetime.datetime(2010, 1, 8, 0, 0, tzinfo=tzutc()),
 'units': 'ugm3',
 'value': '20.6830978393555'},
  {'end_time': datetime.datetime(2010, 1, 9, 23, 0, tzinfo=tzutc()),
 'exposure_type': 'pm25',
 'latitude': '35.9131996',
 'longitude': '-79.0558445',
 'start_time': datetime.datetime(2010, 1, 9, 0, 0, tzinfo=tzutc()),
 'units': 'ugm3',
 'value': '22.1933155059814'}]


In [5]:
"""
CQ3. What pediatric medications are available that act on the protein (gene) targets and biological
  pathways that are activated or inhibited by PM2.5 and ozone exposure?
> Med-BioChem-KS:
  - Get drugs by condition: 
"""
drugs = translator.\
    get_medbiochem().\
    get_drugs_by_condition (conditions=[ "d001249" ])
pprint.pprint (drugs[:10], indent=2)

[ 'Dimethylformamide',
  '3,4-Dihydroxycinnamic Acid',
  '(5e,13e)-9,15-Dihydroxy-11-Oxoprosta-5,13-Dien-1-Oicacid',
  'Glycerol',
  'Carbachol',
  'Imipramine',
  'Clomipramine',
  'Carbamyl-Choline',
  'Acetaminophen',
  'Cyclophosphamide']


In [6]:
"""
CQ2. What protein (gene) targets and biological pathways are activated or inhibited by PM2.5 and ozone exposure?
> Med-BioChem-KS:
  - Get genes and pathways assoicated with a set of conditions. 
"""
conditions = [ 'd001249', 'd003371', 'd001249' ]
genes_paths = translator.\
    get_medbiochem().\
    get_genes_pathways_by_disease (diseases = conditions)
pprint.pprint (genes_paths[:3], indent=2)

[ { 'human  ': True,
    'keggPath': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'pathName': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                '(human)',
    'uniprotGene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'},
  { 'human  ': True,
    'keggPath': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'pathName': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                '(human)',
    'uniprotGene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'},
  { 'human  ': True,
    'keggPath': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'pathName': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                '(human)',
    'uniprotGene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'}]


In [7]:
""" Get genes and pathways involved in exposure to a chemical. We'll use this to determine the 
    effects of PM2.5, or, more generally, of 'Particulate Matter', the MeSH term. """ 
exposures = translator.\
    get_medbiochem().\
    get_exposure_conditions (chemicals = [ 'D052638' ])
pprint.pprint (exposures[:5], indent=2)

[ { 'chemical': 'http://bio2rdf.org/mesh:D052638',
    'gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/NCF1',
    'human': True,
    'pathID': 'hsa04062',
    'pathName': 'path:hsa04062 Chemokine signaling pathway - Homo sapiens '
                '(human)',
    'pathway': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa04062'},
  { 'chemical': 'http://bio2rdf.org/mesh:D052638',
    'gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/NCF1',
    'human': True,
    'pathID': 'hsa04062',
    'pathName': 'path:hsa04062 Chemokine signaling pathway - Homo sapiens '
                '(human)',
    'pathway': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa04062'},
  { 'chemical': 'http://bio2rdf.org/mesh:D052638',
    'gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/COL1A1',
    'human': True,
    'pathID': 'hsa04510',
    'pathName': 'path:hsa04510 Focal adhesion - Homo sapiens (human)',
    'pathway': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa04510'}

In [8]:
print (open ('greentranslator/query/ctd_gene_expo_disease.sparql').read ())

prefix chemgene: <http://chem2bio2rdf.org/ctd/resource/ctd_chem_gene/>
prefix mesh:     <http://bio2rdf.org/mesh:>
prefix ctd:      <http://chem2bio2rdf.org/ctd/resource/>
prefix kegg:     <http://chem2bio2rdf.org/kegg/resource/>
prefix pharmgkb: <http://chem2bio2rdf.org/pharmgkb/resource/>

select ?chemical ?gene ?kegg_pathway ?pathway_name ?pathway_id where {

   values ( ?chemical ) {
      $chemicals
   }

   ?chemgene      ctd:chemicalid       ?chemical ;
                  ctd:interaction      ?interaction ;
                  ctd:interactiontypes ?interaction_types;
                  ctd:gene             ?gene .
 
   ?pharmgkb_gene pharmgkb:Symbol      ?gene ;
                  pharmgkb:UniProt_Id  ?uniprotid .
   
   ?kegg_pathway  kegg:protein         ?uniprotid ;
                  kegg:Pathway_name    ?pathway_name ;
                  kegg:description     ?pathway_descr ;
                  kegg:pathwayid       ?pathway_id ;
                  kegg:ID              ?pathway_nid .


In [9]:
print (open('greentranslator/query/genes_pathways_by_disease.sparql').read ())

prefix kegg:      <http://chem2bio2rdf.org/kegg/resource/>
prefix pharmgkb:  <http://chem2bio2rdf.org/pharmgkb/resource/>
prefix drugbank:  <http://chem2bio2rdf.org/drugbank/resource/>
prefix pubchem:   <http://chem2bio2rdf.org/pubchem/resource/>
prefix ctd:       <http://chem2bio2rdf.org/ctd/resource/>
prefix omim:      <http://chem2bio2rdf.org/omim/resource/>
prefix mesh:      <http://bio2rdf.org/mesh:>

select ?diseaseID ?drugGenericName ?swissProtID ?uniprotGeneID ?geneBankID ?pathwayName ?keggPath where {

    ?keggPath    kegg:protein                ?swissProtID ;
                 kegg:Pathway_name           ?pathwayName .
    ?keggInter   kegg:cid                    ?pubchemCID .

    ?dbInter     drugbank:GeneBank_ID        ?geneBankID ;
                 drugbank:SwissProt_ID       ?swissProtID ;
                 drugbank:gene               ?uniprotGeneID ;
                 drugbank:Name               ?name ;
                 drugbank:DBID               ?drugID .
  
    ?drugID